# SHARPlib -- Lifting Parcels
This is a tutorial notebook on how to conduct parcel lifting operations using SHARPlib. 

SHARPlib has a bit of a different take on parcel lifting relative to existing libraries, allowing for configurable moist adiabats and pseudoadiabats as part of the API. This also means that the API is a little more verbose, since rather than providing a one-size-fits-all function to do this, it instead provides composable functions to achieve a desired end result. If that sounds confusing, don't worry too much about it! You'll soon see how you can use the different building blocks within SHARPlib to do some rather interesting computations.

## Step 0: Imports and Loading Data
Before we can get started, we need to import the modules we would like to use, and load some testing data provided with the repository. The data are stored in the Apache Parquet format, which Pandas can read as long as a compatible Parquet reader is installed (e.g. pyarrow, fastparquet).

The data we are using is from a real weather balloon launch -- 2016-05-24 1900 UTC at Dodge City, Kansas. It is a raw, full resolution dataset with a sample every 1-second during the balloon ascent, containing several thousand vertical levels. This much data can be slow to process in pure-Python (for example, in SHARPpy) so it makes for an excellent example to show how powerful SHARPlib can be. However, because it is a real-world field data source, it isn't exactly clean. It contains some missing values and requires unit conversion, so a convenience function is provided below for data loading. 

This is a good time to tell you that SHARPlib expects you to be responsible for your own data formatting and pre-processing, though it does have internal QC checks for missing values. It expects all data to be in base metric units -- Pascals, Kelvin, meters / second, g / g, etc.

In [17]:
from nwsspc.sharp.calc import parcel
from nwsspc.sharp.calc import thermo
from nwsspc.sharp.calc import layer
import pandas as pd 
import numpy as np 

In [13]:
def load_parquet(filename):
    snd_df = pd.read_parquet(filename)
    snd_df = snd_df[snd_df["vwin"].notna()]
    snd_df = snd_df[snd_df["tmpc"].notna()]
    snd_df = snd_df[snd_df["relh"].notna()]
    snd_df = snd_df[snd_df["pres"] >= 50.0]

    pres = snd_df["pres"].to_numpy().astype('float32')*100.0
    hght = snd_df["hght"].to_numpy().astype('float32')
    tmpk = snd_df["tmpc"].to_numpy().astype('float32')+273.15
    dwpk = snd_df["dwpc"].to_numpy().astype('float32')+273.15
    wdir = snd_df["wdir"].to_numpy().astype('float32')
    wspd = snd_df["wspd"].to_numpy().astype('float32')
    uwin = snd_df["uwin"].to_numpy().astype('float32')
    vwin = snd_df["vwin"].to_numpy().astype('float32')

    # turn into height above ground level
    hght -= hght[0]

    # TO-DO - need a better interface to the API for doing this
    # uwin = np.empty(wspd.shape, dtype="float32")
    # vwin = np.empty(wspd.shape, dtype="float32")
    mixr = thermo.mixratio(pres, dwpk)
    vtmp = thermo.virtual_temperature(tmpk, mixr)

    return {
        "pres": pres, "hght": hght,
        "tmpk": tmpk, "mixr": mixr,
        "vtmp": vtmp, "dwpk": dwpk,
        "wdir": wdir, "wspd": wspd,
        "uwin": uwin, "vwin": vwin
    }

snd_data = load_parquet("../../data/test_snds/ddc.parquet")